adding dependencies


In [1]:
import sys, os
import pandas as pd
import numpy as np


importing modules

In [2]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization,AveragePooling2D
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.utils import np_utils

In [3]:
tf.__version__

'2.2.0'

importing the data

In [4]:
df=pd.read_csv('fer2013.csv')

In [5]:
df

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training
...,...,...,...
35882,6,50 36 17 22 23 29 33 39 34 37 37 37 39 43 48 5...,PrivateTest
35883,3,178 174 172 173 181 188 191 194 196 199 200 20...,PrivateTest
35884,0,17 17 16 23 28 22 19 17 25 26 20 24 31 19 27 9...,PrivateTest
35885,3,30 28 28 29 31 30 42 68 79 81 77 67 67 71 63 6...,PrivateTest


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35887 entries, 0 to 35886
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   emotion  35887 non-null  int64 
 1   pixels   35887 non-null  object
 2   Usage    35887 non-null  object
dtypes: int64(1), object(2)
memory usage: 841.2+ KB


In [7]:
print(df["Usage"].value_counts())


Training       28709
PrivateTest     3589
PublicTest      3589
Name: Usage, dtype: int64


emotions is divided into (0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral) categories<br>
the pixels contains 48*48 coloums of data <br>
usage is basically the use of the data as training, public test or  private test<br>

In [8]:
df.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


declaring the test and train variables
here we declare empty list for the variables

In [9]:
X_train,y_train,X_test,y_test=[],[],[],[]

As the pixel coloumn contains 48 values so we have convert into list <br>
We to it by iterating it through a for loop

In [10]:
for index,row in df.iterrows():
    val=row["pixels"].split(" ")
    try:
        if "Training" in row['Usage']: #-------if value of a usage is training then we apppend the pixels in X_train
            X_train.append(np.array(val,'float32')) # as we have already stored pixels in list "val" so we appendand convert it into float 32
            y_train.append(row['emotion'])
        elif "PublicTest" in row['Usage']: # if usage is given as public test then we append the data in test part of the dataset
            X_test.append(np.array(val,'float32'))
            y_test.append(row['emotion'])
    except:
        print(f"error occured at index :{index} and row:{row}")

In [11]:
#print(val)

In [12]:
print(type(X_train))
print(type(X_test))
print(type(y_train))
print(type(y_test))

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>


As keras only accepts the np array format input so we have to convert all the train and test split into np array 

In [13]:
X_train = np.array(X_train,'float32')
y_train = np.array(y_train,'float32')
X_test = np.array(X_test,'float32')
y_test = np.array(y_test,'float32')

In [14]:
print(type(X_train))
print(type(X_test))
print(type(y_train))
print(type(y_test))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


normalizing the data bw 0 and 1 for better predictions

step-1) Substaracting mean<br>
step2) dividing by stanadrd Deviation

In [15]:
X_train-=np.mean(X_train,axis=0)
X_train/=np.std(X_train,axis=0)

X_test-=np.mean(X_test,axis=0)
X_test/=np.std(X_test,axis=0)

In [16]:
print(X_train)

[[-0.6098866  -0.4592209  -0.40325198 ... -0.08629682 -0.0573744
  -0.41127437]
 [ 0.37236458  0.40461555  0.41102034 ...  1.0387505   0.8913998
   0.8866632 ]
 [ 1.3424892   1.1697278   0.52376574 ... -0.31906524 -0.04455312
   0.47946712]
 ...
 [-0.5613804  -0.44688037 -0.34061563 ...  0.97409266  0.9426849
   0.9248379 ]
 [ 1.2333502   1.3548356   1.1125473  ...  0.30165055  0.28879997
   0.2504193 ]
 [ 0.90593314  1.0093011   1.1376019  ... -1.3794547  -1.2625741
  -0.97116905]]


In [17]:
print(X_test)

[[ 1.6505896   1.7122597   1.774942   ... -0.9225547   0.18117245
   0.80417955]
 [ 0.45219082  0.84190166  1.0658671  ...  0.73589116  0.6640275
   0.5656745 ]
 [-0.6116937   0.02127829 -0.6688345  ... -0.33572    -0.35250947
  -0.32558146]
 ...
 [ 1.6628181   1.7246933   1.7876041  ... -0.84601104 -0.8226578
  -0.87790906]
 [-1.0519217  -1.1350546  -1.0486962  ... -1.4073311  -1.3944598
  -1.4051309 ]
 [-0.7095221  -0.66257447 -0.6941586  ... -0.01678811  0.6386141
   0.80417955]]


In [18]:
print(y_train)

[0. 0. 2. ... 4. 0. 4.]


decralaring some values

In [19]:
num_features = 64 # no.of features of conv2d layer
num_labels = 7
batch_size = 64
epochs = 30
width, height = 48, 48 #pixel 


In [20]:
X_train=X_train.reshape(X_train.shape[0],width,height,1)
X_test=X_test.reshape(X_test.shape[0],width,height,1)

In [21]:
print(X_test)

[[[[ 1.6505896 ]
   [ 1.7122597 ]
   [ 1.774942  ]
   ...
   [-1.2007871 ]
   [-1.4180579 ]
   [ 0.5287383 ]]

  [[ 1.6706394 ]
   [ 1.735452  ]
   [ 1.8093711 ]
   ...
   [-1.18421   ]
   [-1.4165792 ]
   [ 0.06540257]]

  [[ 1.6902738 ]
   [ 1.7594564 ]
   [ 1.835705  ]
   ...
   [-1.0598068 ]
   [-1.4072696 ]
   [-0.02004949]]

  ...

  [[-0.65270513]
   [-0.3880233 ]
   [-0.21744046]
   ...
   [ 1.8131278 ]
   [ 1.7565633 ]
   [ 1.7378885 ]]

  [[-0.43341777]
   [-0.3116194 ]
   [-0.3077701 ]
   ...
   [ 1.5599844 ]
   [ 1.785993  ]
   [ 1.7480594 ]]

  [[-0.3203255 ]
   [-0.33700454]
   [-0.38944486]
   ...
   [-0.9225547 ]
   [ 0.18117245]
   [ 0.80417955]]]


 [[[ 0.45219082]
   [ 0.84190166]
   [ 1.0658671 ]
   ...
   [ 0.5731197 ]
   [ 0.4906877 ]
   [ 0.39366612]]

  [[ 0.34311205]
   [ 0.8348267 ]
   [ 1.1059843 ]
   ...
   [ 0.6423406 ]
   [ 0.5081616 ]
   [ 0.40972537]]

  [[ 0.2202261 ]
   [ 0.7781215 ]
   [ 1.0771161 ]
   ...
   [ 0.7152708 ]
   [ 0.6148504 ]
   [ 0.4847

desinging the cnn

In [22]:
#1st Layer

In [23]:
model=Sequential()

In [24]:
model.add(Conv2D(num_features,kernel_size=(3,3),activation="relu",input_shape=(X_train.shape[1:])))#kernel size is the filter size that we are going to superimpose n images,relu is recifier activation function 
model.add(Conv2D(num_features,kernel_size=(3,3),activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2))) # mqx pool function choosesthe max value from the pool and ouput is fomed with that
model.add(Dropout(0.5)) #drop out somethings so that model doesnt overfit

In [25]:
#2nd layer

In [26]:
model.add(Conv2D(num_features,kernel_size=(3,3),activation="relu"))#kernel size is the filter size that we are going to superimpose n images,relu is recifier activation function 
model.add(Conv2D(num_features,kernel_size=(3,3),activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2))) # mqx pool function choosesthe max value from the pool and ouput is fomed with that
model.add(Dropout(0.5)) #drop out somethings so that model doesnt overfit

3rd layer- in this layer we double the filters

In [27]:
model.add(Conv2D(2*num_features,kernel_size=(3,3),activation="relu"))
model.add(Conv2D(2*num_features,kernel_size=(3,3),activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2))) 

<img src="capture.png" width="400"/>

TO flatten the function

In [28]:
model.add(Flatten()) # reshapes the function to the number of elements contained

In [29]:
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))

In [30]:
model.add(Dense(num_labels, activation='softmax')) # softmax as we have to choose bw the available labels given

In [31]:
#compiling the model

In [32]:
model.compile(loss=categorical_crossentropy,optimizer=Adam(),metrics=["Accuracy"])

In [33]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 46, 46, 64)        640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 44, 44, 64)        36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 22, 22, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 22, 22, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 20, 20, 64)        36928     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 18, 18, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 9, 9, 64)          0

In [34]:
#Training the model
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test),
          shuffle=True)




Epoch 1/30


ValueError: in user code:

    F:\anaconda\lib\site-packages\tensorflow\python\keras\engine\training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    F:\anaconda\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    F:\anaconda\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    F:\anaconda\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    F:\anaconda\lib\site-packages\tensorflow\python\keras\engine\training.py:533 train_step  **
        y, y_pred, sample_weight, regularization_losses=self.losses)
    F:\anaconda\lib\site-packages\tensorflow\python\keras\engine\compile_utils.py:205 __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    F:\anaconda\lib\

In [ ]:
fer_json = model.to_json()
with open("fer.json", "w") as json_file:
    json_file.write(fer_json)
model.save_weights("fer.h5")